In [6]:
%matplotlib widget

In [2]:
import igl
import numpy as np

In [3]:
def segmentationA(p):
    return p[0]+1

def segmentationB(p):
    return  p[0]-1

In [9]:
def partition(V, T, distA, distB):
    import graph_tool.all as GT

    # Use libigl to setup data-structures for visiting neighbors
    # and getting geometry information about mesh.
    TT, _ = igl.tet_tet_adjacency(T)
    BC = igl.barycenter(V, T)
    vol = igl.volume(V, T)
    avg_vol = np.average(vol)
    
    # The first face of a tet is [0,1,2], the second [0,1,3], the third [1,2,3], and the fourth [2,0,3].    
    triangles = np.array([[0,1,2],[0,1,3],[1,2,3],[2,0,3]], dtype=np.int32)
    area = np.zeros(TT.shape,dtype=np.float64)
    for e in range(len(T)):
        area[e] = igl.doublearea(V, T[e][triangles])/2
    
    avg_area = np.average(area/2) # This is not entirely correct, inner faces are counted twice, outer faces only once.
    omega = 3*(avg_area/avg_vol)**(2/3)
            
    g = GT.Graph()
    g.set_directed(True)    
    g.add_vertex(len(T)+2)
    # Last two nodes are used as terminals
    A_idx = len(T)
    B_idx = A_idx +1
    # Create edges from terminals to all tetrahedral elements. 
    for i in range(len(T)):
        g.add_edge(A_idx, i)
        g.add_edge(i, B_idx)
    # Create edges between neighboring tetrahedra elements
    for i in range(len(T)):
        for j in range(4):
            if TT[i,j] != -1:
                g.add_edge(i, TT[i,j])
    # Next we compute capacities for all edges such that: Edges to/from terminals
    # will store the unary costs of our energy functional, and edges between
    # neighboring tetrahedral elements will store the binary costs.
    cap = g.new_edge_property("double")
    for i in range(len(T)):
        e = g.edge(A_idx, i)
        cap[e] =  omega*(vol[i]/avg_vol)*distA(BC[i])                
        e = g.edge(i, B_idx)
        cap[e] = omega*(vol[i]/avg_vol)*distB(BC[i])
    # Create edges between neighboring tetrahedra elements
    for i in range(len(T)):
        for j in range(4):
            if TT[i,j] != -1:
                e = g.edge(i, TT[i,j])
                cap[e] = area[i,j]/avg_area
    g.edge_properties["cap"] = cap    
        
    res = GT.boykov_kolmogorov_max_flow(g, g.vertex(A_idx), g.vertex(B_idx), cap)
    inA = GT.min_st_cut(g, g.vertex(A_idx), cap, res)    
    return np.array([ val for val in inA],dtype=bool)

In [10]:
V = np.array([[0.0,0.0,0.0],[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0],[1.0,1.0,1.0]],dtype=np.float64)

T = np.array([[1,3,2,0],[1,2,3,4]])

inA = partition(V, T, segmentationA, segmentationB)

print(inA)


[ True  True  True False]


In [12]:
T[[1,1]]

array([[1, 2, 3, 4],
       [1, 2, 3, 4]])

In [15]:
print(V[T[1]][1].shape)


(3,)
